## 1. Initial Inspection of the Raw Lending Club Dataset

This notebook performs a comprehensive initial inspection of the raw Lending Club accepted loans dataset (`lc_accepted_loans_full_2007to2018.csv`).

The primary goals are to:
- Understand the dataset's structure, size, and data types.
- Analyze the distribution of the target variable, `loan_status`.
- Identify columns with missing data.
- Get a high-level statistical summary of the numerical features.
- Create a small, stratified sample for quick visual checks.

In [ ]:
import pandas as pd
import os
import re

## Configuration

Here, we define the file paths and key parameters for the analysis.

In [ ]:
# --- Configuration ---
FILE_TO_ANALYZE = 'lc_accepted_loans_full_2007to2018.csv'
SAMPLE_OUTPUT_FILE = 'lc_loans_stratified_sample.csv'
TOTAL_SAMPLES = 100

## Step 1: Load the Dataset

First, we load the raw dataset from the specified CSV file. We'll then display the shape of the data (rows, columns) and view the first few rows to get a feel for the content.

In [ ]:
# Upload 'lc_accepted_loans_full_2007to2018.csv' file

try:
    df = pd.read_csv(FILE_TO_ANALYZE, low_memory=False, on_bad_lines='skip')
    print("Dataset loaded successfully!")
    print(f"Shape of the dataset (rows, columns): {df.shape}")
    display(df.head())
except FileNotFoundError:
    print(f"Error: The file '{FILE_TO_ANALYZE}' was not found.")
    print("Please make sure you've uploaded the file to this Colab session.")

## Step 2: Target Variable Analysis (`loan_status`)

Understanding the distribution of our target variable is crucial. We will count the occurrences of each loan status and calculate its percentage of the total.

In [ ]:
target_col = 'loan_status'

if target_col in df.columns:
    print(f"--- Analyzing Target Variable: {target_col} --- ")
    counts = df[target_col].value_counts()
    percentages = df[target_col].value_counts(normalize=True) * 100

    summary_df = pd.DataFrame({'Count': counts, 'Percentage': percentages.map('{:.2f}%'.format)})
    summary_df = summary_df.reset_index().rename(columns={'index': target_col})

    print("Distribution of the target variable:")
    print(summary_df.to_string(index=False))
else:
    print(f"Warning: Target variable '{target_col}' not found.")

## Step 3: Inferred Statistical Variable Types

Pandas' default data types (`Dtype`) are technical. This step infers the *statistical purpose* of each column (e.g., Numerical, Categorical, Identifier) based on its data type and the number of unique values. This gives us a more intuitive understanding of our features.

In [ ]:
print("--- Inferring Variable Types ---")
analysis_results = []
total_rows = len(df)
for col in df.columns:
    dtype = df[col].dtype
    unique_values = df[col].nunique()

    clean_type = 'Other'

    if pd.api.types.is_numeric_dtype(df[col]):
        if unique_values > total_rows * 0.95:
            clean_type = 'Identifier'
        else:
            clean_type = 'Numerical'
    elif pd.api.types.is_object_dtype(df[col]):
        if unique_values < 50 or (unique_values / total_rows < 0.02 and unique_values > 1):
            clean_type = 'Categorical'
        elif unique_values == total_rows:
            clean_type = 'Identifier'
        else:
            clean_type = 'Text/High Cardinality'

    if 'date' in col.lower() or col.lower().endswith('_d'):
        clean_type = 'Datetime (Potential)'

    emoji_map = {
        'Identifier': '🆔', 'Numerical': '🔢', 'Categorical': '🏷️',
        'Text/High Cardinality': '📄', 'Datetime (Potential)': '📅', 'Other': '❓'
    }
    inferred_type_with_emoji = f"{clean_type} {emoji_map.get(clean_type, '')}".strip()

    analysis_results.append({
        'Variable': col, 'Unique Values': unique_values,
        'Dtype': dtype, 'Inferred Type': inferred_type_with_emoji,
        'Clean Type': clean_type
    })

results_df = pd.DataFrame(analysis_results)
print(results_df.drop(columns=['Clean Type']).to_string(index=False))

type_counts = results_df['Clean Type'].value_counts()
summary_parts = [f"{name}({count})" for name, count in type_counts.items()]
summary_string = "Inferred Types: " + ", ".join(summary_parts)
print(f"\n{summary_string}")

## Step 4: Descriptive Statistics (Numerical)

Here we generate a statistical summary (count, mean, standard deviation, quartiles, etc.) for all numerical columns in the dataset.

In [ ]:
print("--- Descriptive Statistics (Numerical) ---")
# Set display options to show all columns and format numbers
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', '{:.2f}'.format)

display(df.describe().T)

# Reset display options to default
pd.reset_option('display.max_rows')
pd.reset_option('display.float_format')

## Step 5: Missing Value Analysis

This step identifies which columns are missing the most data. We will display the top 50 columns with the highest counts of missing values.

In [ ]:
print("--- Missing Value Analysis ---")
missing_values = df.isnull().sum()
missing_values = missing_values[missing_values > 0].sort_values(ascending=False)

if not missing_values.empty:
    print("Top 50 columns with the most missing values:")
    print(missing_values.head(50))
else:
    print("No missing values found in the dataset. Great!")

## Step 6: Create a Stratified Sample

As a final step in this initial inspection, we'll create a small, 100-row stratified sample based on the primary `loan_status` categories. This file is useful for quick spot-checks and manual inspection without needing to load the full dataset.

In [ ]:
# Define the function to create the sample
def create_stratified_sample(df_to_sample):
    print("\n--- Creating Stratified Sample from DataFrame ---")
    try:
        sampling_plan = {
            'Fully Paid': int(TOTAL_SAMPLES * 0.40),
            'Current': int(TOTAL_SAMPLES * 0.30),
            'Charged Off': int(TOTAL_SAMPLES * 0.30)
        }
        print("Sampling plan:")
        for status, count in sampling_plan.items():
            print(f"- {status}: {count} rows")

        sample_list = []
        for status, count in sampling_plan.items():
            group = df_to_sample[df_to_sample['loan_status'] == status]
            if len(group) < count:
                print(f"Warning: Not enough samples for '{status}'. Taking {len(group)} rows.")
                count = len(group)
            sample = group.sample(n=count, random_state=42)
            sample_list.append(sample)

        final_sample = pd.concat(sample_list)
        final_sample = final_sample.sample(frac=1, random_state=42).reset_index(drop=True)

        final_sample.to_csv(SAMPLE_OUTPUT_FILE, index=False)
        print(f"\nSuccess! A stratified sample of {len(final_sample)} rows has been saved to '{SAMPLE_OUTPUT_FILE}'")
        print("You can find this file in the Colab file browser on the left.")

    except Exception as e:
        print(f"\nAn error occurred during sampling: {e}")

# Call the function using the loaded DataFrame
create_stratified_sample(df)